# Simpson's paradox

## Example from Causality Primer

From [Primer](http://bayes.cs.ucla.edu/PRIMER/primer-ch1.pdf), table 1.1. is

Subpopulations | No Treatment | Treatment 
------------ | ------------- | ------
Male |  234 of 270 recover (87%) | 81 of 87 recover (93%) 
Female | 55 of 80 recover (69%) | 192 of 263 recover (73%)
Total | 289 of 350 (83%)) | 273 of 350 (78%)


We exchange the rows due to keys having to be in alphabetical order for the fake-data-for-learning package (see [this issue](https://github.com/munichpavel/fake-data-for-learning/issues/8)):

Subpopulations | No Treatment | Treatment 
------------ | ------------- | ------
Female | 55 of 80 recover (69%) | 192 of 263 recover (73%)
Male |  234 of 270 recover (87%) | 81 of 87 recover (93%) 
Total | 289 of 350 (83%)) | 273 of 350 (78%)



## Equations for Simpson paradox

Keeping with the above example, we consider the counts above as being derived from a space of counts along dimensions $(\mathrm{RECOVERED}, \mathrm{GENDER}, \mathrm{TREATED})$ of $\mathbb{N}^2 \times \mathbb{N}^2  \times \mathbb{N}^2$:

$$\begin{align*}
u_{000} &= \textrm{Count of non-recovered females who received no treatment} \\
u_{100} &= \textrm{Count of recovered females who received no treatment} \\
u_{010} &= \textrm{Count of non-recovered males who received no treatment} \\
u_{110} &= \textrm{Count of recovered males who received no treatment} \\
u_{001} &= \textrm{Count of non-recovered females who received treatment} \\
u_{101} &= \textrm{Count of recovered females who received treatment} \\
u_{011} &= \textrm{Count of non-recovered males who received treatment} \\
u_{111} &= \textrm{Count of recovered males who received treatment}
\end{align*}$$

To denote sums along one or more of the count dimensions of RECOVERED, GENDER, TREATED, we substitute the dimension along which we sum by a $+$. So $u_{00+}=147$ is the count of recovered females, regardless of treatment. Hence we can re-write the above table as 

Subpopulations | No Treatment | Treatment 
:----------: | :-----------: | :----: 
Female |  $u_{100}$ of $u_{+00}$ recover (ratio $\frac{u_{100}}{u_{+00}}$) | $u_{101}$ of $u_{+01}$ recover (ratio $\frac{n_{101}}{n_{+01}})$ 
Male | $u_{110}$ of $u_{+10}$  recover (ratio $\frac{u_{110}}{u_{+10}})$ | $u_{111}$ of $u_{+11}$ recover (ratio $\frac{u_{111}}{u_{+11}})$
Total |  $u_{1+0}$ of $u_{++0}$  recover $\left(\frac{u_{1+1}}{u_{++0}}\right)$ |  $n_{1+1}$ of $u_{++1}$  recover (ratio $\frac{u_{1+1}}{u_{++1}})$ 


## As contingency tables

For computation, the $u_{ijk}$ lend themselves to representing experiment outcomes grouped by sub-populations as contigency tables; see [Lectures on Algebraic Statistics](http://math.berkeley.edu/~bernd/owl.pdf), Section 1.1]. 

In this example, the contingency table is a 2x2x2 matrices of event counts. Using [xarray](http://xarray.pydata.org/en/stable/), we can define this matrix in a convenient form for defining, accessing and performing operations (e.g. the marginal sums of the $+$ notation).

In [1]:
import numpy as np
import xarray as xr

from fake_data_for_learning import utils as ut

In [2]:
U = [
    [
        [25, 71],
        [36, 6]
        
    ], 
    [
         [55, 192],
        [234, 81]
    ]
       
]
gender_vals = ['female', 'male']
treated_vals = [0, 1]
recovered_vals = [0, 1]
counts = xr.DataArray(
    U, 
    dims=('recovered', 'gender', 'treated'),
    coords=[recovered_vals, gender_vals, treated_vals]
)

print("Check against counts from contingency table above:\n")
events = [
    dict(recovered=1, gender='female', treated=0),
    dict(recovered=1, gender='female', treated=1),
    dict(recovered=1, gender='male'),
    dict(recovered=1, gender='male'),

]

for event in events:
    marginal_counts = counts.sel(**event)
    print(f'Counts for event(s) {event}: \n{marginal_counts.values}')

Check against counts from contingency table above:

Counts for event(s) {'recovered': 1, 'gender': 'female', 'treated': 0}: 
55
Counts for event(s) {'recovered': 1, 'gender': 'female', 'treated': 1}: 
192
Counts for event(s) {'recovered': 1, 'gender': 'male'}: 
[234  81]
Counts for event(s) {'recovered': 1, 'gender': 'male'}: 
[234  81]


## As conditional probabilities

Note that the ratio $\frac{u_{100}}{u_{+00}}$ can be interpreted as the (empirical) probability of recovery given that the patient was female and received no treatment, or

$$\begin{align*}
P(R=1|G=0, T=0) = \frac{u_{100}}{u_{+00}} = \frac{p_{100}}{p_{+00}},
\end{align*}$$

where $p_{ijk} = u_{ijk} / u_{+++}$ are the usual empirical probabilities.

We can thus translate Simpson's paradox into equations and inequalities in $u_{ijk}$ and $p_{ijk}$ and their marginals (i.e. with $+$s among the indices).




Then the statement that the subpopulations receiving treatment recover better in the above example becomes the inequalities

$$\begin{align*}
\frac{P(R=1|G=0, T=1)}{P(R=1 | G=0, T=0)} = \frac{\frac{p_{101}}{p_{+01}}}{\frac{p_{100}}{p_{+00}}}> 1\\
\frac{P(R=1|G=1, T=1)}{P(R=1|G=1, T=0)}= \frac{\frac{p_{111}}{p_{+11}}}{\frac{p_{110}}{p_{+10}}} > 1 
\end{align*}$$

or, assuming all probabilities are non-zero,

$$\begin{align*}
p_{101}p_{+00} -p_{100}p_{+01} > 0 \\
p_{111}p_{+10} -p_{110}p_{+11} > 0 \tag{SPR}
\end{align*}$$

which we call the *subpopulation ratio* (SPR) inequalities.

The statement that the treated group recovers less often on the entire population than the non-treated one is

$$\begin{equation*}
\frac{P(R=1|T=1)}{P(R=1|T=0)} = \frac{\frac{p_{1+1}}{p_{++1}}}{\frac{p_{1+0}}{p_{++0}}} < 1
\end{equation*}$$

or 

$$\begin{equation*}
p_{1+1}p_{++0} - p_{1+0}p_{++1}  < 0  \tag{TPR}
\end{equation*}$$

which we call the *total population ratio* (TPR) inequality.



### More examples

#### Better recovery with treatment for female subpopulation , all else the same recovery

In the following example the male and overall populations experience no difference between treatment and no-treatment groups, but the female population recovers at a much higher rate with treatment.

Subpopulations | No Treatment | Treatment
------------ | ------------- | ----------
Female |4 of 10 recover (40%) | 22 of 30 recover (73%)
Male | 27 of 30 recover (90%) | 9 of 10 recover (90%) 
Total |31 of 40 recover (78%) | 31 of 40 (78%) 

Again, assuming all probabilies are non-zero, the sub-population and total population (in)equalities are 

$$\begin{align*}
p_{101}p_{+00} -p_{100}p_{+01} > 0 \\
p_{111}p_{+10} -p_{110}p_{+11} = 0 
\end{align*}$$

$$\begin{equation*}
p_{1+1}p_{++0} - p_{1+0}p_{++1}  = 0
\end{equation*}$$

## References

* JUDEA PEARL, MADELYN GLYMOUR, NICHOLAS P. JEWELL, [CAUSAL INFERENCE IN STATISTICS: A PRIMER](http://bayes.cs.ucla.edu/PRIMER/primer-ch1.pdf)
* Mathias Drton, Bernd Sturmfels, Seth Sullivant, [Lectures on Algebraic Statistics](http://math.berkeley.edu/~bernd/owl.pdf)
* Aleksandra B. Slavkovic, Stephen E. Fienberg, [Algebraic geometry of 2 × 2 contingency tables](http://personal.psu.edu/abs12/Research/Papers/slavfienPistone2009.pdf)

## Mini-example counts

with xarray.

In [3]:
U = [[
        [6, 8],
        [3, 1]
    
    ], [
        [4, 22],
        [27, 9]
    ]]
gender_vals = [0, 1]
treated_vals = [0, 1]
recovered_vals = [0, 1]
counts = xr.DataArray(
    U,
    dims=("recovered", "gender", "treated"),
    coords=[recovered_vals, gender_vals, treated_vals]
)

events = [
    dict(recovered=0, gender=0, treated=0),  # not-recovered, female, not-treated event counts
    dict(recovered=1, gender=0, treated=0),  # recovered, female, not-treated event counts
    dict(gender=1, treated=0),  # male, not treated event counts
    dict(recovered=1)  # recovered event counts
]

for event in events:
    marginal_counts = counts.sel(**event)
    print(f'Counts for event(s) {event}: \n{marginal_counts.values}')

Counts for event(s) {'recovered': 0, 'gender': 0, 'treated': 0}: 
6
Counts for event(s) {'recovered': 1, 'gender': 0, 'treated': 0}: 
4
Counts for event(s) {'gender': 1, 'treated': 0}: 
[ 3 27]
Counts for event(s) {'recovered': 1}: 
[[ 4 22]
 [27  9]]


## ... to conditional probability matrices and odds-ratios

Conditional probabilities like $P(R=1|G=0, T=0) = \frac{u_{100}}{u_{+00}}$ can be readily calculated with the xarray syntax.

In [4]:
counts_over_recovered = counts.sum(dim='recovered')  # the u_{+jk}
cpt_on_gender_treatment = counts / counts_over_recovered 

cond_prob = cpt_on_gender_treatment.sel(recovered=1, gender=0, treated=1)
print(f'Probability of recovery given treated female: {cond_prob.data}')
cond_prob = cpt_on_gender_treatment.sel(recovered=0, gender=0, treated=1)
print(f'Probability of no-recovery given treated female: {cond_prob.data}')

Probability of recovery given treated female: 0.7333333333333333
Probability of no-recovery given treated female: 0.26666666666666666


Now $P(R=1| T=0) = \frac{u_{1+0}}{u_{++0}}$

In [5]:
counts_over_gender = counts.sum(dim='gender')
counts_over_gender_recovered = counts_over_gender.sum(dim='recovered')
counts_over_gender_recovered

<xarray.DataArray (treated: 2)>
array([40, 40])
Coordinates:
  * treated  (treated) int64 0 1

In [6]:
cpt_on_treatment = counts_over_gender / counts_over_gender_recovered
print(f'Probability of recovery given not-treated: {cpt_on_treatment.sel(recovered=1, treated=0).data}')
print(f'Probability of recovery given treated: {cpt_on_treatment.sel(recovered=1, treated=1).data}')

Probability of recovery given not-treated: 0.775
Probability of recovery given treated: 0.775


Odds-ratios are also natural to compute.

In [7]:
# Female odd-ratio > 1 with treatment vs no-treatment
odds_ratio_female = (
    cpt_on_gender_treatment.sel(recovered=1, gender=0, treated=1) / 
    cpt_on_gender_treatment.sel(recovered=1, gender=0, treated=0)
)
print(f'Odds ratio for females, treatment vs no-treatment: {odds_ratio_female.data}')

Odds ratio for females, treatment vs no-treatment: 1.833333333333333


In [8]:
# Male population odds-ratio for treatment vs no-treatment
odds_ratio_male = (
    cpt_on_gender_treatment.sel(recovered=1, gender=1, treated=1) / 
    cpt_on_gender_treatment.sel(recovered=1, gender=1, treated=0)
)
print(f'Odds ratio for males, treatment vs no-treatment: {odds_ratio_male.data}')

Odds ratio for males, treatment vs no-treatment: 1.0


In [9]:
# Total population odd-ratio = 1 for treatment vs no-treatment
odds_ratio_total = (
    cpt_on_treatment.sel(recovered=1,  treated=1) / 
    cpt_on_treatment.sel(recovered=1, treated=0)
)
print(f'Odds ratio for total population, treatment vs no-treatment: {odds_ratio_total.data}')

Odds ratio for total population, treatment vs no-treatment: 1.0


### Exercise: No-simpson

Show that if all sub-population counts $u_{ijk}$ are all equal to a constant $u_{i} > 0$ for all $j,k$, then Simpson's paradox cannot occur.

### Exercise: Coordinate change about a Simpson point

Assume $p_{ijk}$ as before ($i,j,k \in \{0,1\}$) such that the sub-population and total population (in)equalities are 

$$\begin{align*}
p_{101}p_{+00} -p_{100}p_{+01} > 0 \\
p_{111}p_{+10} -p_{110}p_{+11} = 0 
\end{align*}$$

$$\begin{equation*}
p_{1+1}p_{++0} - p_{1+0}p_{++1}  = 0
\end{equation*}$$

For $epsilon > 0$, consider the coordinate change

$$\begin{align*}
p_{101} \mapsto p_{101} + \epsilon \\
p_{100} \mapsto p_{100} - \epsilon
\end{align*}$$

where $\epsilon$ is chosen small enough that $p_{101} + \epsilon < 1$ and $p_{100} - \epsilon > 0$.

This change makes the first subpopulation inequality further from zero, potentially increasing the ''paradox,'' as the treated female subpopulation fares even better.

What happens to the other two (in-)equalities? Give your answer in terms of the original $p_{ijk}$ and $\epsilon$.

### Exercise: Coordinate change about a Simpson point, II

Do the same as above but for 

$$\begin{align*}
p_{101} \mapsto p_{101} + \epsilon \\
p_{100} \mapsto p_{100} - \epsilon
\end{align*}$$

where $\epsilon$ is chosen small enough that $p_{101} + \epsilon < 1$ and $p_{100} - \epsilon > 0$.

This change makes the first subpopulation inequality further from zero, potentially ''increasing the paradox,'' as the treated female subpopulation fares even better under treatment than with the original $p_{ijk}$.

What happens to the other two (in-)equalities? Give your answer in terms of the original $p_{ijk}$ and $\epsilon$.

## Odds-ratios in terms of counts

Let $\alpha_0$ and $\alpha_1$ be the (conditional) odds-ratios for the female, resp. male subpopulations, respectively, and let $\alpha$ be the total (conditional) odds-ratio.


### Exercise: Simpson odds-ratios in terms of counts

In terms of odds-ratios, Simpson's paradox roughly occurs when odds ratio(s) of one or more subpopulations is less than (resp. greater than) 1, while the trend is reversed in the total population, or no trend exists, i.e. odds ratios are 1.

Show that 

$$\begin{align*}
\alpha_0 &= \frac{ u_{101} u_{+00} }{ u_{100} u_{+10} } \\
\alpha_1 &= \frac{ u_{111} u_{+10} }{ u_{110} u_{+11} } \\
\alpha &= \frac{ u_{1+1} u_{++0} }{ u_{1+0} u_{++1} }
\end{align*}$$

### Exercise: more extreme Simpson case of female subpopulation examples above

Consider the count transformation

$$\begin{align*}
\phi_{101}: u_{101} &\mapsto u_{101} + \epsilon \\
\phi_{111}: u_{111} &\mapsto u_{111} + \nu
\end{align*}$$

And assume that the gender x treatment sub-population counts remain fixed, i.e. the $u_{+jk}$ do not change (by changing the non-recovery subpopulation counts, e.g. $u_{100} \mapsto u_{100} - \epsilon$.)

Find a $\phi_{110}: u_{110} \mapsto \phi_{110}(u_{110})$ in terms of $\nu$ that preserves the odds-ratio $\alpha_1$

